# Data Wrangling with WeRateDogs®

#### By: Justin W Brown

WeRateDogs® is a phenomenon turned company that allows users to "tweet" photos of dogs and include a rating, generally on a 10 point scale.  This rating serves no purpose other than to indicate the user's thoughts and opinions about the subject and is often offered in a comedic manner.  Ignoring the limits of the 10 point scale, many will award many more points, such as 1000 out of 10.  This has become a very successful social media phenomenon and serves as the target of our data wrangling efforts.

## The Data Sources

This effort involves obtaining data from three sources in three different formats:
1. A .csv file provided by Udacity containing initial cleaning efforts on 5000 tweets from WeRateDogs®. (file: twitter-archive-enhanced.csv)
2. A .tsv file that is available for download at a given url and is obtained programmatically. (file: image-predictions.tsv)
3. Using the Twitter api, the application accesses the Twitter data to obtain additional information on each of the 5000 Tweets (if available). (file: tweet_json.txt)

Additionally, I sought and located an additional data source (described below) that is publically accessible containing common names of dogs.

## The Wrangling

### twitter-archive-enhanced.csv

The first data source contains information about 5000 tweets.  The file is provided for us and downloaded and stored locally for use.  Some effort has been made to parse the tweets to extract usable data, but all records are not pristine.  Following are a few issues identified and corrected:

- Immediately it ws discovered that the filename provided by Udacity contained errors.  The filename communicated for use utilized the underscore character (i.e. twitter_archive_enhanced.csv).  The correct filename was located and utilized which contained dash characters (i.e. twitter-archive-enhanced.csv).
- The data contained retweets that replicated intial ratings. These records are identified by the presence of a retweeted "tweet_id" (original tweets contained NaN for this field) and removed from the data set.
- The timestamp is stored as a string and has been converted to a proper timestamp format.
- The source filed contained html which has been removed leaving a clean indicator showing what app or website the tweet originated from.
- The expanded_urls field contains multiple urls, some of which are clearly not accurate sources.  These have been screened and removed, as well as any duplicates.  In the end, it was found that each record had exactly 0 or 1 proper url.  This url is stored in a new field named url.
- The fact the that the ratings are not structured allows for great social media interaction, however, when analyzing the data, it is better to have a standardized rating. For this reason, it has been decided that all numerators and denominators would be normalized to a denominator of 10.  In addition, the ratio of the numerator and denominator has been calculated and stored in the data for potential ease of analyzes later.
- Some fields, unfortunately have little or no usefule data.  These fields are removed from the cleaned data set.
- The name field contains some values that are not representative of actual dog names.  Initially it was noted through observation that all of the inappropriate values are all lower case.  While this is a good way to identify some of the data cleanliness issues, I opted to additionally go on a search for a better way to clean this field.  By searching the internet I found that the AKC has a website listing dog names.  Deciding that this is an authoritative source, I utilized the "inspect page" functionality in the web browser and identified that the data being displayed across the multiple screens on the AKC website comes from a file named "data.json" located at the following url: https://www.akc.org/wp-content/themes/akc/assets/dog-name-finder/data.json  I programmatically accessed this file and extracted all of the dog names on the AKC website.  I then compared this list of "Official" names with the names int he provided data set, hoping to find more inappropriate values.  Unfortunately, the AKC list is not as exhastive as initially hoped.  In the end, the data is cleaned by eliminating values that are in all lower case and values containing the word "None" which was also identified visually.

### image-predictions.tsv

This data souce is obtained programmatically from a url which was provided.  Being a .tsv file, some changes are required to read and parse it correctly.  This file contains information about each dog and the top three results of running the image through an image prediction algorithm to attempt to identify the photo content, ideally resulting in the identification of the breed of the dog int he photo.  Each of the top three results is paired with a value representing the confidence level that the result of the image prediction is accurate on a 0-100 scale.  Additionally there is a field indicating whether the result derived represents a dog breed or something else.  Ultimately, this data set is deemed clean enough for use, although some fields are not deemed useful for analysis and are removed from the final data set.

### tweet_json.txt

This data is obtained using the Tweepy Python library to access the Twitter api.  Each Tweet in the provided data set is queried and certain interesting and potentially useful data is returned and saved as a .json file.  Primarily, the information obtained is to be the counts of the number of "likes" and retweets each tweet have received.  These are both indicators of popularity for each original Tweet.  In addition, I noticed a field which purported to contain the coordinates of the Tweet origin.  Thinking this may be an interesting angle for analysis, I return this field as well.  Once obtained, the following has been observed and handled:

- Unfortunately, the coordinates field contained almost no data and is removed from the final data set.
- When obtaining the data, I chose to make the "tweet_id" the index for the DataFrame.  This is not consistent with the other files and is moved to the first column for consistency and ease of handling.
- Because the data obtained will be constantly updated in Twitter as new retweets and likes occur, I thought it useful to store a timestamp informing future analysts of when the data was obtained.  Pulling the data at other times may provide interesting data for trend analysis.  This timestamp is obtained from the creation date of the tweet_json.txt file.

### Final Data Set

During the cleaning process it was decided that an appropriate struction for this data would be to separate the information about the Tweets and the information about the Dogs into two separate data structures.  This is accomplished by merging portions the data sets to create two new DataFrams: new_tweet_clean and new_dog_clean.  Final tidiness is accomplished by dropping the fields deemed non-useful and ensuring all data types are corrected to reflect the data contained in the field (i.e. Integer, Float, Boolean, etc.).  Additionally, since very few records are replies (which may or may not have additional catalogueable data) and the information regarding the Tweet to which the record related is therefore not extremely useful, it has been decided that a single new boolean value would store the indicator making the analyst aware of the fact that the record represent a reply, and the three fields representing the "reply to" information (which overwhelmingly contained NaN) have been removed.

The final two data sets are stored in files named "twitter_archive_master.csv" and "dog_archive_master.csv" in preparation for analysis.